In [82]:
import pandas as pd
from collections import defaultdict

# ---------- Display (optional) ----------
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 0)
pd.set_option('display.max_colwidth', None)

# ---------- Global State ----------

portfolio_state = {
    'cash': 200,                     # constant after reset
    'remaining': 200,                # evolves with trades
    'quantities': defaultdict(int),  # {ticker: quantity}
    'cost_basis': defaultdict(float),# {ticker: absolute cost basis}
    'avg_price': defaultdict(float), # {ticker: average entry price}
    'realized_pnl': 0.0,
    'last_price': {},                # {ticker: last seen price}
    'portfolio_df': None             # accumulator
}

# Track trade numbers per ticker
trade_tracker = {}  # {ticker: trade_number}
next_trade_number = 1  # Global counter for next new trade

COLUMNS = [
    'Date','Ticker','Buy/Sell','Position Taken','Current Position','Cash','Buyable/Sellable',
    'Quantity Buy','Remaining','Current Quantity','Price',
    'Avg Price','Cost Basis','Position Value PV',
    'PnL (Long) Unrealized','PnL (Short) Unrealized','Pnl Unrealized','PnL Unrealized Value',
    'PV (Long)','PV (Short)','Open Position','Open PV',
    'Total PV','Equity: Total PV + Remaining','PnL Realized at Point of Time','PnL Realized Cummulative','Total PnL Overall (Unrealized+Realized)',
    'Daily PnL (Unrealized+Realized)','Liquidation Price','Take Profit','Stop Loss', 
    'Last Day Pnl / Daily $', 'Daily %', 'Cumulative %', 'Performance', 
    'Trade No. (Position - Trade no. - Current Quantity)', 'Total Trades', 'Win/Loss', 'Win Rate', 'Win:Loss Ratio', 
    'Trades/Month', 'Absolute Quantity Counts', 'Most Traded Symbol', 'Least Traded',
    'Avg Losing PnL', 'Avg Winning PnL', 'Most Profitable', 'Least Profitable', 'Max Drawdown',
    'Total Gain', 'Average Gain', 'Biggest Investment', 'Average Position', 'Average Holdings'
]

# ---------- Lifecycle ----------

def reset_portfolio(initial_cash=200):
    global portfolio_state
    portfolio_state = {
        'cash': initial_cash,
        'remaining': initial_cash,
        'quantities': defaultdict(int),
        'cost_basis': defaultdict(float),
        'avg_price': defaultdict(float),
        'realized_pnl': 0.0,
        'last_price': {},
        # 'asset_type_mapping': get_asset_type_mapping(),
        'max_investment_history': defaultdict(float),
        'highest_traded_volume': None,  # Track global max(quantity*price)
        'lowest_traded_volume': None,   # Track global min(quantity*price)
        'position_open_period': {},  # Track when each ticker position was first opened {ticker: period_number}
        'cumulative_holding_sum': 0.0,  # Sum of all holding periods for closed positions
        'closed_positions_count': 0,  # Number of positions that have been closed
        'current_period': 0,  # Track current period/row number
        'previous_realized_pnl': 0.0,
        'portfolio_df': pd.DataFrame(columns=COLUMNS)
    }
    trade_tracker = {}
    next_trade_number = 1

def get_portfolio_df():
    return portfolio_state['portfolio_df'].copy()

# ---------- Helpers ----------

def normalize_quantity(q):
    """
    Accepts:
      - numeric: 10, -5
      - strings: "-(-10)" -> 10,  "(10)" -> 10, "-10" -> -10, "  -(-5)  " -> 5
    """
    if isinstance(q, (int, float)):
        return float(q)
    s = str(q).strip().replace(' ', '')
    if s.startswith('-(') and s.endswith(')'):
        inner = s[2:-1]
        return -float(inner)
    if s.startswith('(') and s.endswith(')'):
        inner = s[1:-1]
        return float(inner)
    return float(s)

def get_or_create_trade_number(ticker, old_q, new_q, action):
    """
    Determine trade number for a ticker based on position state.
    - If old_q == 0 and new_q != 0: Opens new trade (assign new trade number)
    - If old_q != 0 and new_q != 0: Continues existing trade (keep same trade number)
    - If old_q != 0 and new_q == 0: Closes existing trade (keep same trade number, mark as closed)
    - If old_q == 0 and new_q == 0: No trade
    """
    global next_trade_number
    ticker = str(ticker).upper()
    
    action_lower = str(action).lower()
    
    # If action is 'hold', check if position exists
    if action_lower == 'hold':
        if ticker in trade_tracker and old_q != 0:
            # Position exists but holding, return existing trade number
            return trade_tracker[ticker]
        else:
            return None
    
    # If opening a new position (old_q == 0, new_q != 0)
    if old_q == 0 and new_q != 0:
        # Assign new trade number
        trade_number = next_trade_number
        trade_tracker[ticker] = trade_number
        next_trade_number += 1
        return trade_number
    
    # If closing a position (old_q != 0, new_q == 0)
    if old_q != 0 and new_q == 0:
        # Return existing trade number, then remove from tracker
        trade_number = trade_tracker.get(ticker)
        if ticker in trade_tracker:
            del trade_tracker[ticker]
        return trade_number
    
    # If maintaining/changing position (old_q != 0, new_q != 0)
    if old_q != 0 and new_q != 0:
        # Continue existing trade
        return trade_tracker.get(ticker)
    
    # No position (old_q == 0, new_q == 0)
    return None

def format_trade_string(action, current_position, trade_number, new_q):
    """
    Format trade string: "Buy/Sell/Short - #TradeNo Trade - Quantity" or "... - 0 - close"
    Includes Buy/Sell action and uses current_position instead of position
    """
    if trade_number is None:
        return "No Buy/Sell"
    
    action_str = str(action)
    pos = str(current_position)
    
    # Format quantity - use absolute value since position already explains it
    if new_q == 0:
        quantity_str = "0 - Close"
    else:
        # Use absolute value for quantity
        quantity_str = str(int(abs(new_q)))
    
    # Include Buy/Sell in the output
    return f" {pos.capitalize()} - {action_str.capitalize()} - #{trade_number} Trade - {quantity_str}"

# ---------- Core Single-Trade Calculations ----------

def calculate_cash_single():
    # Cash is constant after initialization
    return portfolio_state['cash']

def calculate_remaining_single(action, position, price, q_in, old_quantity, old_cost_basis):
    """
    Remaining update (prev = prior Remaining, p = current price, q = shares):
    - Buy Long:   prev - p*q
    - Sell Long:  prev + p*q
    - Sell Short: prev - p*q
    - Buy Short:  prev + [ initial + (initial - final) ]
                  where initial = avg * close_qty, final = p * close_qty
    
    Uses previous avg price (from old_cost_basis / old_quantity) for closing calculations.
    
    Position flips automatically when quantity crosses zero:
    - Long selling more than owned: closes long, opens short with excess
    - Short buying more than owed: covers short, opens long with excess
    """
    rem = portfolio_state['remaining']
    a = str(action).lower()
    qty = abs(q_in) if q_in < 0 else q_in
    if qty == 0 or a == 'hold':
        return rem

    # Previous avg price (0 if no position)
    prev_avg = abs(old_cost_basis / old_quantity) if old_quantity else 0.0

    if a == 'buy':
        if old_quantity < 0:
            # Buy Short (cover up to held short) using prev_avg for initial
            cover = min(qty, abs(old_quantity))
            if cover > 0:
                initial = prev_avg * cover
                final = price * cover
                delta = initial + (initial - final)
                rem += delta
            # Any excess turns into Buy Long at market
            excess = qty - cover
            if excess > 0:
                rem -= price * excess
        else:
            # Buy Long
            rem -= price * qty
        return rem

    if a == 'sell':
        if old_quantity > 0:
            # Sell Long up to held long
            close = min(qty, old_quantity)
            if close > 0:
                rem += price * close
            # Any excess turns into Sell Short at market
            excess = qty - close
            if excess > 0:
                rem -= price * excess
        else:
            # Sell Short (open/increase short)
            rem -= price * qty
        return rem

    return rem

def calculate_current_quantity_single(ticker, action, position, q_in, old_quantity):
    """
    Quantity change depends on action:
    - Buy: increases quantity (moves longward)
    - Sell: decreases quantity (moves shortward)
    
    Position flips automatically when quantity crosses zero:
    - Long selling more than owned → flips to short
    - Short buying more than owed → flips to long
    """
    a = str(action).lower()
    qty = abs(q_in) if q_in < 0 else q_in

    if a == 'hold':
        new_q = old_quantity
    elif a == 'buy':
        new_q = old_quantity + qty  # Naturally flips short→long if buying excess
    elif a == 'sell':
        new_q = old_quantity - qty  # Naturally flips long→short if selling excess
    else:
        new_q = old_quantity

    portfolio_state['quantities'][ticker] = new_q
    return new_q

def calculate_avg_price_and_cost_basis_single(ticker, action, position, price, q_in, old_quantity, new_quantity, old_cost_basis):
    """
    Maintain absolute Cost Basis and Avg Price for net position.
    Conventions:
    - Long: Cost Basis = dollars spent on current net long shares
    - Short: Cost Basis = dollars received from current net short shares (short proceeds)
    - Avg Price = abs(Cost Basis / Quantity) when quantity != 0
    - Crossing 0: prior side closes, new side starts fresh
    """
    a = str(action).lower()
    qty = abs(q_in) if q_in < 0 else q_in
    cb = old_cost_basis

    if a == 'hold':
        pass

    elif a == 'buy':
        if old_quantity >= 0:
            # adding/opening long
            cb = cb + qty * price
        else:
            # buying to cover short
            to_cover = min(qty, abs(old_quantity))
            if qty > to_cover:
                # fully cover then open long with residual
                open_long = qty - to_cover
                cb = open_long * price
            else:
                # still short; proportionally reduce short proceeds
                if abs(old_quantity) > 0:
                    remaining_short = abs(old_quantity) - to_cover
                    cb = cb * (remaining_short / abs(old_quantity)) if remaining_short > 0 else 0.0

    elif a == 'sell':
        if old_quantity > 0:
            # selling from long
            sell_qty = qty
            if sell_qty < old_quantity:
                remaining_long = old_quantity - sell_qty
                cb = cb * (remaining_long / old_quantity)
            elif sell_qty == old_quantity:
                cb = 0.0
            else:
                # flipped to short: close long then open short with extra
                open_short = sell_qty - old_quantity
                cb = open_short * price
        else:
            # short selling or increasing short
            cb = cb + qty * price

    if new_quantity != 0:
        avg_price = abs(cb / new_quantity)
    else:
        avg_price = 0.0
        cb = 0.0

    portfolio_state['cost_basis'][ticker] = cb
    portfolio_state['avg_price'][ticker] = avg_price
    return avg_price, cb

def calculate_realized_pnl_at_point_of_time(ticker, action, position, price, q_in, old_quantity):
    """
    Calculate realized PnL at point of time using (price - avg_entry) * q formula.
    This is independent and dynamic, not dependent on cumulative calculations.
    Returns PnL for this specific closing action only.
    """
    a = str(action).lower()
    pos = str(position).lower()
    
    # Read old avg price from state (before it gets updated)
    prev_avg_price = portfolio_state['avg_price'][ticker]
    
    realized_pnl_point = None

    # Long position: calculate realized PnL when selling
    if pos == 'long' and a == 'sell' and old_quantity > 0:
        qty = abs(q_in) if q_in < 0 else q_in
        # Only realize on shares actually closed (min of qty and owned)
        closed = min(qty, old_quantity)
        if closed > 0 and prev_avg_price > 0:
            realized_pnl_point = (price - prev_avg_price) * closed

    # Short position: calculate realized PnL when buying/covering
    elif pos == 'short' and a == 'buy' and old_quantity < 0:
        qty = abs(q_in) if q_in < 0 else q_in
        # Only realize on shares actually covered (min of qty and owed)
        closed = min(qty, abs(old_quantity))
        if closed > 0 and prev_avg_price > 0:
            realized_pnl_point = (prev_avg_price - price) * closed

    return realized_pnl_point

def calculate_realized_pnl_cumulative(ticker, action, position, price, q_in, old_quantity):
    """
    Realized PnL on closing legs - cumulative across ALL tickers.
    - Closing long by selling: (sell_price - avg_entry) * shares_closed
    - Covering short by buying: (avg_entry - cover_price) * shares_closed
    
    Uses previous avg price from state (reads directly before updating).
    Long position: calculate realized PnL when action is 'sell'
    Short position: calculate realized PnL when action is 'buy'
    
    IMPORTANT: When position flips (selling more than owned or buying more than owed),
    only the closed portion generates realized PnL, not the excess.
    """
    realized = portfolio_state['realized_pnl']
    a = str(action).lower()
    pos = str(position).lower()
    
    # Read old avg price from state (before it gets updated)
    prev_avg_price = portfolio_state['avg_price'][ticker]

    # Long position: calculate realized PnL when selling
    if pos == 'long' and a == 'sell' and old_quantity > 0:
        qty = abs(q_in) if q_in < 0 else q_in
        # Only realize on shares actually closed (min of qty and owned)
        # If selling more than owned, only the owned shares generate realized PnL
        closed = min(qty, old_quantity)
        if closed > 0 and prev_avg_price > 0:
            realized += (price - prev_avg_price) * closed

    # Short position: calculate realized PnL when buying/covering
    if pos == 'short' and a == 'buy' and old_quantity < 0:
        qty = abs(q_in) if q_in < 0 else q_in
        # Only realize on shares actually covered (min of qty and owed)
        # If buying more than owed, only the owed shares generate realized PnL
        closed = min(qty, abs(old_quantity))
        if closed > 0 and prev_avg_price > 0:
            realized += (prev_avg_price - price) * closed

    portfolio_state['realized_pnl'] = realized
    return realized

# ---------- Derived per-trade ----------

def position_value_from_position(position, new_quantity, price):
    pos = str(position).lower()
    if pos == 'short':
        # show positive PV for shorts
        return abs(new_quantity) * price
    # long/hold → normal signed PV
    return new_quantity * price

def pnl_unrealized_components(new_quantity, price, avg_price):
    if new_quantity > 0 and avg_price > 0:
        long_u = (price - avg_price) * new_quantity
    else:
        long_u = 0.0

    if new_quantity < 0 and avg_price > 0:
        short_u = (avg_price - price) * abs(new_quantity)
    else:
        short_u = 0.0

    return long_u, short_u, (long_u + short_u)

def open_positions_str():
    parts = []
    for t, q in portfolio_state['quantities'].items():
        if q != 0:
            ticker_upper = str(t).upper()
            parts.append(f"{ticker_upper} {q}")
    return ", ".join(parts) if parts else "None"

def open_pv_str(current_ticker, current_price, current_position):
    parts = []
    for t, q in portfolio_state['quantities'].items():
        if q != 0:
            cb = portfolio_state['cost_basis'][t]
            avg = portfolio_state['avg_price'][t]
            
            if t == current_ticker:
                p = current_price
            else:
                p = portfolio_state['last_price'].get(t, 0.0)
            
            if q > 0 and avg > 0:
                long_u = (p - avg) * q
                ticker_pv = cb + long_u
            elif q < 0 and avg > 0:
                short_u = (avg - p) * abs(q)
                ticker_pv = cb + short_u
            else:
                ticker_pv = 0.0
            
            ticker_upper = str(t).upper()
            parts.append(f"{ticker_upper} {ticker_pv}")
    return ", ".join(parts) if parts else "None"

def calculate_pv_for_current_ticker(current_ticker, current_price, current_position, new_q, avg_p, cb):
    """
    Calculate PV (Long) and PV (Short) ONLY for the current ticker being traded.
    PV = Cost Basis + Unrealized PnL
    """
    pos = str(current_position).lower()
    
    if pos == 'long' and new_q > 0 and avg_p > 0:
        long_u = (current_price - avg_p) * new_q
        pv_long = cb + long_u
        return pv_long, 0.0
    elif pos == 'short' and new_q < 0 and avg_p > 0:
        short_u = (avg_p - current_price) * abs(new_q)
        pv_short = cb + short_u
        return 0.0, pv_short
    else:
        return 0.0, 0.0

def calculate_total_pv_all_tickers(current_ticker, current_price, current_position):
    """
    Calculate Total PV = sum of all long PVs + sum of all short PVs
    across all tickers in the portfolio (cumulative).
    PV = Cost Basis + Unrealized PnL for each position
    """
    total_long_pv = 0.0
    total_short_pv = 0.0
    
    for t, q in portfolio_state['quantities'].items():
        if q != 0:
            # Get cost basis and avg price for this ticker
            cb = portfolio_state['cost_basis'][t]
            avg = portfolio_state['avg_price'][t]
            
            # Determine current price
            if t == current_ticker:
                p = current_price
            else:
                p = portfolio_state['last_price'].get(t, 0.0)
            
            # Calculate unrealized PnL and PV
            if q > 0 and avg > 0:  # Long position
                long_u = (p - avg) * q
                ticker_pv_long = cb + long_u
                total_long_pv += ticker_pv_long
            elif q < 0 and avg > 0:  # Short position
                short_u = (avg - p) * abs(q)
                ticker_pv_short = cb + short_u
                total_short_pv += ticker_pv_short
    
    return total_long_pv + total_short_pv

def open_pnl_unrealized_str(current_ticker, current_price, current_position):
    parts = []
    for t, q in portfolio_state['quantities'].items():
        if q != 0:
            avg = portfolio_state['avg_price'][t]
            
            if t == current_ticker:
                p = current_price
            else:
                p = portfolio_state['last_price'].get(t, 0.0)
            
            if q > 0 and avg > 0:
                ticker_unrealized = (p - avg) * q
            elif q < 0 and avg > 0:
                ticker_unrealized = (avg - p) * abs(q)
            else:
                ticker_unrealized = 0.0
            
            ticker_upper = str(t).upper()
            parts.append(f"{ticker_upper} {ticker_unrealized}")
    
    return ", ".join(parts) if parts else "None"

def calculate_liquidation_price(current_position, new_q, avg_p):
    """
    Calculate liquidation price based on current position at that point in time.
    - Long position: liquidation at price = 0
    - Short position: liquidation at price = 2 * avg_price (100% loss = liquidated)
    - Hold position or quantity = 0: None (no position to liquidate)
    """
    pos = str(current_position).lower()
    
    # Check that we actually have a position (quantity != 0)
    if new_q == 0:
        return None
    
    if pos == 'long' and avg_p > 0:
        # Long position: liquidation at price = 0
        return 0.0
    elif pos == 'short' and avg_p > 0:
        # Short position: liquidation at price = 2 * avg_price
        # (e.g., short at 100, if price goes to 200 = 100% loss = liquidated)
        return 2.0 * avg_p
    else:
        # Hold position or invalid avg_p
        return None

def calculate_take_profit(current_position, new_q, avg_p, take_profit_pct=0.20):
    """
    Calculate Take Profit price based on current position.
    - Long: Avg Entry Price * (1 + Percentage)
    - Short: Avg Entry Price * (1 - Percentage)
    """
    if new_q == 0 or avg_p <= 0:
        return None
    
    pos = str(current_position).lower()
    
    if pos == 'long':
        # Long: Take Profit = Avg Price * (1 + Percentage)
        return avg_p * (1 + take_profit_pct)
    elif pos == 'short':
        # Short: Take Profit = Avg Price * (1 - Percentage)
        return avg_p * (1 - take_profit_pct)
    else:
        return None

def calculate_stop_loss(current_position, new_q, avg_p, stop_loss_pct=0.10):
    """
    Calculate Stop Loss price based on current position.
    - Long: Avg Entry Price * (1 - Percentage) [unrealized = -cost basis * percentage]
    - Short: Avg Entry Price * (1 + Percentage) [vice versa of TP]
    """
    if new_q == 0 or avg_p <= 0:
        return None
    
    pos = str(current_position).lower()
    
    if pos == 'long':
        # Long: Stop Loss = Avg Price * (1 - Percentage)
        return avg_p * (1 - stop_loss_pct)
    elif pos == 'short':
        # Short: Stop Loss = Avg Price * (1 + Percentage) [vice versa of TP]
        return avg_p * (1 + stop_loss_pct)
    else:
        return None

def calculate_trade_win_loss(trade_string, realized_pnl_at_point):
    """
    Calculate if a closed trade is a win or loss based on trade string.
    Win: realized PnL at point of time > 0
    Loss: realized PnL at point of time <= 0
    Only applies when trade closes (trade_string contains "- close")
    Uses the independent point-of-time realized PnL calculation
    """
    # Check if trade is closing based on trade string format
    if trade_string == "No Buy/Sell" or "- Close" not in trade_string:
        return None
    
    # Trade is closing, check the realized PnL at point of time
    if realized_pnl_at_point > 0:
        return "Win"
    elif realized_pnl_at_point <= 0:
        return "Loss"
    else:
        return None

def calculate_win_rate(previous_df, current_win_loss):
    """
    Calculate win rate at this point in time.
    Win Rate = (Win values / Total non-None values) * 100
    Counts all previous Win/Loss values plus current one
    """
    # Get all previous Win/Loss values
    if len(previous_df) > 0:
        previous_win_loss = previous_df['Win/Loss'].tolist()
        # Add current win_loss to the list
        all_win_loss = previous_win_loss + [current_win_loss]
    else:
        all_win_loss = [current_win_loss]
    
    # Count non-None values
    non_none_values = [v for v in all_win_loss if v is not None]
    total_non_none = len(non_none_values)
    
    # Count Win values
    win_count = sum(1 for v in non_none_values if v == "Win")
    
    # Calculate win rate
    if total_non_none > 0:
        win_rate = (win_count / total_non_none) * 100
        return win_rate
    else:
        return None

def calculate_win_loss_ratio(previous_df, current_win_loss):
    """
    Calculate win:loss ratio at this point in time.
    Win:Loss Ratio = "win_count:loss_count" format (e.g., "1:0", "2:1", "0:1")
    """
    # Get all previous Win/Loss values
    if len(previous_df) > 0:
        previous_win_loss = previous_df['Win/Loss'].tolist()
        # Add current win_loss to the list
        all_win_loss = previous_win_loss + [current_win_loss]
    else:
        all_win_loss = [current_win_loss]
    
    # Count non-None values
    non_none_values = [v for v in all_win_loss if v is not None]
    
    # Count Win and Loss values
    win_count = sum(1 for v in non_none_values if v == "Win")
    loss_count = sum(1 for v in non_none_values if v == "Loss")
    
    # Return ratio in "win_count:loss_count" format
    return f"{win_count}:{loss_count}"

def calculate_trades_per_month(previous_df, current_date, current_trade_string):
    """
    Calculate number of open trades in the current month.
    Returns format: "count (Month Name)" or just count with month name
    """
    from datetime import datetime
    
    if current_date is None:
        return None
    
    # Parse current date and get month name
    try:
        if isinstance(current_date, str):
            # Try common date formats
            for fmt in ['%m/%d/%Y', '%Y-%m-%d', '%d/%m/%Y', '%m-%d-%Y']:
                try:
                    current_dt = datetime.strptime(current_date, fmt)
                    break
                except:
                    continue
            else:
                return None
        else:
            current_dt = pd.to_datetime(current_date)
        
        month_name = current_dt.strftime('%B')  # Full month name (e.g., "October")
        current_month_year = (current_dt.year, current_dt.month)
    except:
        return None
    
    # Get all rows including current
    all_data = []
    if len(previous_df) > 0:
        for _, row in previous_df.iterrows():
            all_data.append({
                'Date': row.get('Date'),
                'Trade String': row.get('Trade No. (Position - Trade no. - Current Quantity)', '')
            })
    all_data.append({
        'Date': current_date,
        'Trade String': current_trade_string
    })
    
    # Find open trades in current month
    open_trade_nums = set()
    
    for row_data in all_data:
        row_date = row_data['Date']
        trade_str = row_data['Trade String']
        
        if row_date is None or trade_str == "No Buy/Sell":
            continue
        
        # Check if row is in current month
        try:
            if isinstance(row_date, str):
                for fmt in ['%m/%d/%Y', '%Y-%m-%d', '%d/%m/%Y', '%m-%d-%Y']:
                    try:
                        row_dt = datetime.strptime(row_date, fmt)
                        break
                    except:
                        continue
                else:
                    continue
            else:
                row_dt = pd.to_datetime(row_date)
            
            row_month_year = (row_dt.year, row_dt.month)
            if row_month_year != current_month_year:
                continue
            
            # Extract trade number
            import re
            match = re.search(r'#(\d+)', str(trade_str))
            if match:
                trade_num = int(match.group(1))
                # Check if trade is closed
                if "- close" not in str(trade_str):
                    open_trade_nums.add(trade_num)
                else:
                    # Remove if closed
                    open_trade_nums.discard(trade_num)
        except:
            continue
    
    count = len(open_trade_nums)
    return f"{count} ({month_name})"

def calculate_most_least_traded(previous_df, current_ticker, current_qty_buy):
    """
    Calculate most and least traded symbols based on cumulative absolute quantity counts.
    Sums absolute values from "Quantity Buy" column per ticker.
    Returns: (absolute_quantity_counts_str, most_traded, least_traded)
    Most Traded: all tickers ordered by quantity descending
    Least Traded: all tickers ordered by quantity ascending
    """
    from collections import defaultdict
    
    # Track cumulative absolute quantities for each ticker from "Quantity Buy" column
    ticker_quantities = defaultdict(float)
    
    # Process previous rows - sum absolute quantities from "Quantity Buy" column
    if len(previous_df) > 0:
        for _, row in previous_df.iterrows():
            row_ticker = str(row.get('Ticker', '')).upper()
            row_qty_buy = row.get('Quantity Buy', 0)
            if row_ticker and row_qty_buy != 0:
                try:
                    qty = float(row_qty_buy)
                    ticker_quantities[row_ticker] += abs(qty)
                except:
                    pass
    
    # Add current quantity buy
    if current_ticker and current_qty_buy != 0:
        try:
            ticker_quantities[current_ticker] += abs(float(current_qty_buy))
        except:
            pass
    
    if not ticker_quantities:
        return None, "None", "None"
    
    # Sort tickers by quantity (descending), then alphabetically for ties
    sorted_tickers_desc = sorted(ticker_quantities.items(), key=lambda x: (-x[1], x[0]))
    
    # Calculate absolute quantity counts string (ordered by quantity descending)
    abs_counts = [f"{ticker} {int(qty)}" for ticker, qty in sorted_tickers_desc]
    abs_counts_str = ", ".join(abs_counts) if abs_counts else "None"
    
    # Most Traded: All tickers ordered by quantity descending
    most_traded_list = [ticker for ticker, qty in sorted_tickers_desc]
    most_traded_str = ", ".join(most_traded_list) if most_traded_list else "None"
    
    # Least Traded: All tickers ordered by quantity ascending
    sorted_tickers_asc = sorted(ticker_quantities.items(), key=lambda x: (x[1], x[0]))
    least_traded_list = [ticker for ticker, qty in sorted_tickers_asc]
    least_traded_str = ", ".join(least_traded_list) if least_traded_list else "None"
    
    return abs_counts_str, most_traded_str, least_traded_str

def calculate_avg_losing_winning_pnl(previous_df, current_realized_pnl_at_point):
    """
    Calculate average losing and winning PnL from realized PnL at point of time.
    Avg Losing PnL: Average of all realized PnL at point of time where PnL < 0
    Avg Winning PnL: Average of all realized PnL at point of time where PnL > 0
    """
    # Get all previous realized PnL at point of time values
    if len(previous_df) > 0:
        previous_pnl_values = previous_df['PnL Realized at Point of Time'].tolist()
        # Add current value to the list
        all_pnl_values = previous_pnl_values + [current_realized_pnl_at_point]
    else:
        all_pnl_values = [current_realized_pnl_at_point]
    
    # Filter for losing PnL (< 0) and winning PnL (> 0)
    losing_pnl = [pnl for pnl in all_pnl_values if pnl is not None and pnl < 0]
    winning_pnl = [pnl for pnl in all_pnl_values if pnl is not None and pnl > 0]
    
    # Calculate averages
    avg_losing_pnl = sum(losing_pnl) / len(losing_pnl) if len(losing_pnl) > 0 else 0.0
    avg_winning_pnl = sum(winning_pnl) / len(winning_pnl) if len(winning_pnl) > 0 else 0.0
    
    return avg_losing_pnl, avg_winning_pnl

def calculate_most_least_profitable(previous_df, current_ticker, current_realized_pnl_at_point):
    """
    Calculate most and least profitable tickers based on realized PnL at point of time.
    Most Profitable: Ticker where Max(value where realized pnl > 0)
    Least Profitable: Ticker where Min(value where realized pnl > 0)
    Only considers winning trades (PnL > 0)
    Returns format: "TICKER PnL_Value" or "TICKER1 PnL1, TICKER2 PnL2" if multiple
    """
    # Track all winning trades (ticker, pnl) pairs where pnl > 0
    winning_trades = []
    
    # Process previous rows
    if len(previous_df) > 0:
        for _, row in previous_df.iterrows():
            row_ticker = str(row.get('Ticker', '')).upper()
            row_pnl = row.get('PnL Realized at Point of Time', 0)
            if row_ticker and row_pnl is not None and row_pnl > 0:
                winning_trades.append((row_ticker, row_pnl))
    
    # Add current PnL if it's a winning trade
    if current_ticker and current_realized_pnl_at_point is not None and current_realized_pnl_at_point > 0:
        winning_trades.append((current_ticker, current_realized_pnl_at_point))
    
    if not winning_trades:
        return "None", "None"
    
    # Find overall max and min PnL values across all winning trades
    max_pnl = max(pnl for _, pnl in winning_trades)
    min_pnl = min(pnl for _, pnl in winning_trades)
    
    # Find tickers with max PnL (most profitable)
    most_profitable_trades = [(ticker, pnl) for ticker, pnl in winning_trades if pnl == max_pnl]
    most_profitable_list = [f"{ticker} {pnl}" for ticker, pnl in sorted(most_profitable_trades)]
    most_profitable = ", ".join(most_profitable_list) if most_profitable_list else "None"
    
    # Find tickers with min PnL (least profitable)
    least_profitable_trades = [(ticker, pnl) for ticker, pnl in winning_trades if pnl == min_pnl]
    least_profitable_list = [f"{ticker} {pnl}" for ticker, pnl in sorted(least_profitable_trades)]
    least_profitable = ", ".join(least_profitable_list) if least_profitable_list else "None"
    
    return most_profitable, least_profitable

def calculate_max_drawdown(current_total_pv):
    """
    Calculate Max Drawdown = (MAX(Total PV from row 2 to current) - MIN(Total PV from row 2 to current)) / MAX(Total PV from row 2 to current)
    
    For each row:
    - Row 1: 0 (no calculation)
    - Row 2: (MAX(Total PV row 2) - MIN(Total PV row 2)) / MAX(Total PV row 2) = 0
    - Row 3: (MAX(Total PV rows 2-3) - MIN(Total PV rows 2-3)) / MAX(Total PV rows 2-3)
    - Row 4: (MAX(Total PV rows 2-4) - MIN(Total PV rows 2-4)) / MAX(Total PV rows 2-4)
    - etc.
    
    Note: Calculation starts from row 2 (skips first row).
    
    Args:
        current_total_pv: Current Total PV value for this row
    
    Returns:
        Max Drawdown as float (0 for first row, calculated value for subsequent rows)
    """
    # Check if this is the first row (portfolio_df is empty)
    if portfolio_state['portfolio_df'] is None or portfolio_state['portfolio_df'].empty:
        return 0.0
    
    # Get Total PV values starting from row 2 (skip first row)
    # We need to get Total PV from row 2 onwards, plus the current row
    total_pv_values = []
    
    # Get Total PV values from previous rows (starting from row 2, skipping row 1)
    if 'Total PV' in portfolio_state['portfolio_df'].columns:
        # Get all Total PV values starting from row 2 (index 1 onwards)
        all_pv_values = portfolio_state['portfolio_df']['Total PV'].tolist()
        if len(all_pv_values) > 0:
            # Skip first row (index 0), take from row 2 onwards (index 1 onwards)
            total_pv_values.extend(all_pv_values[1:])
    
    # Add current row's Total PV (this will be row 2 or later)
    total_pv_values.append(current_total_pv)
    
    # Calculate MAX and MIN
    if not total_pv_values:
        return 0.0
    
    max_pv = max(total_pv_values)
    min_pv = min(total_pv_values)
    
    if max_pv == 0:
        return 0.0
    
    # Calculate Max Drawdown: (MAX - MIN) / MAX
    max_drawdown = (max_pv - min_pv) / max_pv
    
    return max_drawdown

def update_max_investment_history(ticker, price, quantity_buy, action, old_quantity):
    """
    Update the historical maximum investment for a ticker.
    Uses price * quantity_buy (the value of the trade).
    Tracks both 'buy' actions (long positions) and 'sell' actions (short positions).
    Only updates when opening/expanding positions (entry points).

    Args:
        ticker: Ticker symbol
        price: Price of the trade
        quantity_buy: Quantity in the trade (positive for buy, positive for sell)
        action: Trade action ('buy', 'sell', 'hold')
        old_quantity: Quantity before this trade
    """
    action_lower = str(action).lower()
    qty = abs(quantity_buy) if quantity_buy < 0 else quantity_buy

    # Track maximum investment for:
    # 1. Buy actions (opening/expanding long positions)
    # 2. Sell actions that open/expand short positions (when old_quantity <= 0)
    if action_lower == 'buy':
        # Buy action: opening/expanding long position
        investment_value = price * qty
        current_max = portfolio_state['max_investment_history'].get(ticker, 0.0)
        if investment_value > current_max:
            portfolio_state['max_investment_history'][ticker] = investment_value

    elif action_lower == 'sell':
        # Sell action: check if it's opening/expanding a short position
        # Short position opens when old_quantity <= 0 and we're selling
        if old_quantity <= 0:
            # This sell opens or expands a short position (entry point for short)
            investment_value = price * qty
            current_max = portfolio_state['max_investment_history'].get(ticker, 0.0)
            if investment_value > current_max:
                portfolio_state['max_investment_history'][ticker] = investment_value
        # If old_quantity > 0, this is closing a long position, not opening a short
        # So we don't track it as an entry point


def calculate_biggest_investment():
    """
    Calculate Biggest Investment = max(price * quantity_buy) historically,
    for every symbol (including closed positions), ordered by size.

    Uses historical maximum (price * quantity_buy) from buy actions (long) 
    and sell actions that open short positions.
    Shows all tickers that have ever had positions, even if currently closed.
    
    Returns:
        String formatted as "SYMBOL: MAX_INVESTMENT, SYMBOL: MAX_INVESTMENT" 
        (ordered by max investment descending)
        or "None" if no historical investments
    """
    # Get all tickers that have historical investment records (including closed positions)
    positions = []
    for ticker, max_investment in portfolio_state['max_investment_history'].items():
        if max_investment > 0:  # Only include tickers with historical max > 0
            positions.append({
                'ticker': ticker.upper(),
                'max_investment': max_investment
            })
    
    if not positions:
        return "None"
    
    # Sort by historical maximum investment (descending) - biggest first
    positions.sort(key=lambda x: x['max_investment'], reverse=True)
    
    # Format as "SYMBOL: MAX_INVESTMENT, SYMBOL: MAX_INVESTMENT"
    parts = [f"{pos['ticker']}: {int(pos['max_investment'])}" for pos in positions]
    
    return ", ".join(parts)

def calculate_average_position(current_df, current_action, current_position_value_pv):
    """
    Calculate Average Position = avg(position value) when Buy/Sell == "buy"
    Formula: IF(Buy/Sell == "buy", AVERAGEIF(Buy/Sell == "buy" up to current row, Position Value PV), "0")

    Args:
        current_df: DataFrame up to current row (excluding current row)
        current_action: Current row's Buy/Sell action
        current_position_value_pv: Current row's Position Value PV

    Returns:
        Average as float (or "0" string if current action is not "buy")
    """
    action_lower = str(current_action).lower()

    if action_lower != 'buy':
        return "0"

    # Get all Position Value PV values where Buy/Sell == "buy" up to and including current row
    # We need to include the current row's PV in the calculation
    buy_pvs = []

    # Add PVs from previous rows where Buy/Sell == "buy"
    if not current_df.empty and 'Buy/Sell' in current_df.columns and 'Position Value PV' in current_df.columns:
        buy_rows = current_df[current_df['Buy/Sell'].str.lower() == 'buy']
        if not buy_rows.empty:
            buy_pvs.extend(buy_rows['Position Value PV'].tolist())

    # Add current row's PV if it's a buy
    if action_lower == 'buy':
        buy_pvs.append(current_position_value_pv)

    # Calculate average
    if buy_pvs:
        avg = sum(buy_pvs) / len(buy_pvs)
        return round(avg, 4)  # Match the format shown (4 decimal places)
    else:
        return "0"
    
def calculate_holdings():
    """
    Calculate Holdings = COUNTA(UNIQUE Tickers, where Current Quantity ≠ 0)
    Count of unique tickers that have non-zero current quantity
    """
    unique_tickers = set()
    for ticker, qty in portfolio_state['quantities'].items():
        if qty != 0:
            unique_tickers.add(ticker.upper())
    return len(unique_tickers)

def update_traded_volume_history(price, quantity_buy, action, current_position):
    """
    Update the historical highest and lowest traded volume.
    Uses quantity_buy * price for trades where action != 'hold'.
    We track based on the action (buy/sell), not the resulting position.
    
    Args:
        price: Price of the trade
        quantity_buy: Quantity in the trade (quantity_buy)
        action: Trade action ('buy', 'sell', 'hold')
        current_position: Current position after the trade ('long', 'short', 'hold')
    """
    action_lower = str(action).lower()
    
    # Track if action is 'buy' or 'sell' (not 'hold')
    # This ensures we track trades even if they close positions (resulting in 'hold')
    if action_lower in ['buy', 'sell']:
        qty = abs(quantity_buy) if quantity_buy < 0 else quantity_buy
        traded_volume = price * qty  # quantity_buy * price
        
        # Update highest traded volume
        if portfolio_state['highest_traded_volume'] is None:
            portfolio_state['highest_traded_volume'] = traded_volume
        else:
            if traded_volume > portfolio_state['highest_traded_volume']:
                portfolio_state['highest_traded_volume'] = traded_volume
        
        # Update lowest traded volume
        if portfolio_state['lowest_traded_volume'] is None:
            portfolio_state['lowest_traded_volume'] = traded_volume
        else:
            if traded_volume < portfolio_state['lowest_traded_volume']:
                portfolio_state['lowest_traded_volume'] = traded_volume
    # If action is 'hold', don't update - keep last known values

def get_highest_traded_volume():
    """
    Get Highest Traded Volume = max(quantity*price) if position != hold
    Returns the historical maximum traded volume across all trades.
    If no trades have been recorded, returns 0.
    """
    if portfolio_state['highest_traded_volume'] is None:
        return 0
    return int(portfolio_state['highest_traded_volume'])

def get_lowest_traded_volume():
    """
    Get Lowest Traded Volume = min(quantity*price) if position != hold
    Returns the historical minimum traded volume across all trades.
    If no trades have been recorded, returns 0.
    """
    if portfolio_state['lowest_traded_volume'] is None:
        return 0
    return int(portfolio_state['lowest_traded_volume'])

# ---------- Average Holding Days Calculation ----------

def track_position_opening(ticker, current_period):
    """
    Track when a position is first opened for a ticker.
    Only tracks if position is being opened (not already tracked).
    """
    if ticker not in portfolio_state['position_open_period']:
        portfolio_state['position_open_period'][ticker] = current_period

def detect_closed_positions(old_quantities, new_quantities, current_period):
    """
    Detect positions that were closed (quantity went from non-zero to zero).
    Returns list of closed tickers with their holding periods.
    """
    closed_positions = []
    
    # Check all tickers that were in old_quantities
    for ticker in set(list(old_quantities.keys()) + list(new_quantities.keys())):
        old_qty = old_quantities.get(ticker, 0)
        new_qty = new_quantities.get(ticker, 0)
        
        # Position closed if it went from non-zero to zero
        if old_qty != 0 and new_qty == 0:
            # Calculate holding period
            if ticker in portfolio_state['position_open_period']:
                open_period = portfolio_state['position_open_period'][ticker]
                holding_period = current_period - open_period + 1
                closed_positions.append({
                    'ticker': ticker,
                    'holding_period': holding_period
                })
                # Remove from tracking since it's closed
                del portfolio_state['position_open_period'][ticker]
    
    return closed_positions

def update_average_holding_days(closed_positions):
    """
    Update cumulative average holding days when positions are closed.
    
    Args:
        closed_positions: List of dicts with 'ticker' and 'holding_period'
    """
    if closed_positions:
        for closed in closed_positions:
            holding_period = closed['holding_period']
            # Add to cumulative sum
            portfolio_state['cumulative_holding_sum'] += holding_period
            # Increment count of closed positions
            portfolio_state['closed_positions_count'] += 1

def calculate_average_holding_days():
    """
    Calculate Average Holding Days = (sum of all holding periods) / (number of closed positions)
    Returns the cumulative average holding days for all closed positions.
    """
    if portfolio_state['closed_positions_count'] == 0:
        return None  # No positions closed yet
    
    avg = portfolio_state['cumulative_holding_sum'] / portfolio_state['closed_positions_count']
    return round(avg, 3)  # Round to 3 decimal places

# ---------- Main entry per trade ----------

def process_trade(ticker, action, position, price, quantity_buy, date=None, take_profit_pct=0.20, stop_loss_pct=0.10):
    ticker = str(ticker).strip().upper()
    
    q_in = normalize_quantity(quantity_buy)

    old_q = portfolio_state['quantities'][ticker]
    old_cb = portfolio_state['cost_basis'][ticker]

    cash = calculate_cash_single()
    new_remaining = calculate_remaining_single(action, position, price, q_in, old_q, old_cb)

    new_q = calculate_current_quantity_single(ticker, action, position, q_in, old_q)
    
    current_position = 'short' if new_q < 0 else 'long' if new_q > 0 else 'hold'
    
    # Calculate realized PnL at point of time (independent calculation)
    realized_pnl_at_point = calculate_realized_pnl_at_point_of_time(
        ticker, action, position, price, q_in, old_q
    )
    
    realized_pnl_cumulative = calculate_realized_pnl_cumulative(
        ticker, action, position, price, q_in, old_q
    )
    
    avg_p, cb = calculate_avg_price_and_cost_basis_single(
        ticker, action, position, price, q_in, old_q, new_q, old_cb
    )

    buyable_sellable = (new_remaining / price) if price > 0 else 0.0
    pv = position_value_from_position(current_position, new_q, price)
    long_u, short_u, u_total = pnl_unrealized_components(new_q, price, avg_p)
    open_pos = open_positions_str()
    open_pv = open_pv_str(ticker, price, current_position)
    open_pnl = open_pnl_unrealized_str(ticker, price, current_position) 

    pv_long_current, pv_short_current = calculate_pv_for_current_ticker(ticker, price, current_position, new_q, avg_p, cb)
    
    total_pv = calculate_total_pv_all_tickers(ticker, price, current_position)
    total_pv_equity = total_pv + new_remaining
    
    # Calculate Total PnL Overall = Equity - Initial Cash
    total_pnl_overall = total_pv_equity - cash
    
    # Calculate Daily PnL = Today's Total PnL Overall - Yesterday's Total PnL Overall
    # Get previous row's Total PnL Overall if it exists, otherwise 0 (first trade)
    previous_df = portfolio_state['portfolio_df']
    if len(previous_df) > 0:
        previous_total_pnl_overall = previous_df.iloc[-1]['Total PnL Overall (Unrealized+Realized)']
        # Get yesterday's equity for Daily % calculation
        yesterday_equity = previous_df.iloc[-1]['Equity: Total PV + Remaining']
    else:
        previous_total_pnl_overall = 0.0
        yesterday_equity = cash  # Use initial cash as baseline for first trade
    
    daily_pnl = total_pnl_overall - previous_total_pnl_overall
    
    # Calculate Daily % = (Today's Equity - Yesterday's Equity) / Yesterday's Equity * 100
    if yesterday_equity > 0:
        daily_pct = ((total_pv_equity - yesterday_equity) / yesterday_equity) * 100
    else:
        daily_pct = None  # Avoid division by zero
    
    # Calculate Cumulative % = ((Equity / Initial Cash) - 1) * 100
    if cash > 0:
        cumulative_pct = ((total_pv_equity / cash) - 1) * 100
    else:
        cumulative_pct = None  # Avoid division by zero

    # Performance is the same as Cumulative %
    performance = cumulative_pct
    
    # Calculate Number of Trades = Track trades per ticker
    # Get trade number for this ticker
    trade_number = get_or_create_trade_number(ticker, old_q, new_q, action)
    
    # Format trade string
    trade_string = format_trade_string(action, current_position, trade_number, new_q)
    
    # Calculate total trades (max trade number that has been assigned)
    if trade_string == "No Buy/Sell":
        # Use previous total if available
        if len(previous_df) > 0:
            last_total = previous_df.iloc[-1].get('Total Trades', "No Buy/Sell")
            if isinstance(last_total, str) and "Trades" in last_total:
                total_trades_str = last_total
            else:
                total_trades_str = "No Buy/Sell"
        else:
            total_trades_str = "No Buy/Sell"
    else:
        # Total trades is the highest trade number opened so far
        total_trades_str = f"{next_trade_number - 1} Trades"
    
    # Calculate Liquidation Price based on current position and quantity
    liquidation_price = calculate_liquidation_price(current_position, new_q, avg_p)
    
    # Calculate Take Profit and Stop Loss
    take_profit = calculate_take_profit(current_position, new_q, avg_p, take_profit_pct)
    stop_loss = calculate_stop_loss(current_position, new_q, avg_p, stop_loss_pct)
    
    # Calculate Win/Loss for closed trades (using trade_string to check if trade closed)
    win_loss = calculate_trade_win_loss(trade_string, realized_pnl_at_point)
    
    # Calculate Win Rate at this point
    win_rate = calculate_win_rate(previous_df, win_loss)
    
    # Calculate Win:Loss Ratio at this point
    win_loss_ratio = calculate_win_loss_ratio(previous_df, win_loss)
    
     # Calculate Trades/Month
    trades_per_month = calculate_trades_per_month(previous_df, date, trade_string)
    
    # Calculate Most/Least Traded
    abs_quantity_counts, most_traded_symbol, least_traded_symbol = calculate_most_least_traded(
        previous_df, ticker, q_in
    )
    
    # Calculate Avg Losing PnL and Avg Winning PnL
    avg_losing_pnl, avg_winning_pnl = calculate_avg_losing_winning_pnl(
        previous_df, realized_pnl_at_point
    )
    
    # Calculate Most/Least Profitable
    most_profitable, least_profitable = calculate_most_least_profitable(
        previous_df, ticker, realized_pnl_at_point
    )

    # Calculate Max Drawdown
    max_drawdown = calculate_max_drawdown(total_pv)

    # Calculate Total Gain (same as PnL Realized Cummulative)
    total_gain = realized_pnl_cumulative
    
    # Calculate Average Gain = Total Gain / Total Trades
    # Total Trades = next_trade_number - 1 (number of trades that have been opened)
    total_trades_count = next_trade_number - 1
    if total_trades_count > 0:
        avg_gain = total_gain / total_trades_count
    else:
        avg_gain = 0.0  # No trades opened yet
    
    # Update historical maximum investment
    update_max_investment_history(ticker, price, q_in, action, old_q)
    
    # Calculate Biggest Investment
    biggest_investment = calculate_biggest_investment()

    # Calculate new columns: Average Position, Holdings, Assets
    current_df = portfolio_state['portfolio_df'].copy() if portfolio_state['portfolio_df'] is not None else pd.DataFrame()
    avg_position = calculate_average_position(current_df, action, pv)
    holdings = calculate_holdings()

    # Calculate Highest/Lowest Traded Volume
    highest_traded_volume = get_highest_traded_volume()
    lowest_traded_volume = get_lowest_traded_volume()
    
    # Calculate Average Holding Days
    average_holding_days = calculate_average_holding_days()

    row = {
        'Date': date,
        'Ticker': ticker,
        'Buy/Sell': action.capitalize(),
        'Position Taken': position.capitalize(),
        'Current Position': current_position.capitalize(),
        'Cash': cash,
        'Buyable/Sellable': buyable_sellable,
        'Quantity Buy': q_in,
        'Remaining': new_remaining,
        'Current Quantity': new_q,
        'Price': price,
        'Avg Price': avg_p,
        'Cost Basis': cb,
        'Position Value PV': pv,
        'PnL (Long) Unrealized': long_u,
        'PnL (Short) Unrealized': short_u,
        'Pnl Unrealized': open_pnl,
        'PnL Unrealized Value': u_total,
        'PV (Long)': pv_long_current,
        'PV (Short)': pv_short_current,
        'Open Position': open_pos,
        'Open PV': open_pv,
        'Total PV': total_pv,
        'Equity: Total PV + Remaining': total_pv_equity,
        'PnL Realized at Point of Time': realized_pnl_at_point,
        'PnL Realized Cummulative': realized_pnl_cumulative,
        'Total PnL Overall (Unrealized+Realized)': total_pnl_overall,
        'Daily PnL (Unrealized+Realized)': daily_pnl,
        'Liquidation Price': liquidation_price,
        'Take Profit': take_profit,
        'Stop Loss': stop_loss,
        'Last Day Pnl / Daily $': daily_pnl,
        'Daily %': daily_pct,
        'Cumulative %': cumulative_pct,
        'Performance': performance,
        'Trade No. (Position - Trade no. - Current Quantity)': trade_string,
        'Total Trades': total_trades_str,
        'Win/Loss': win_loss,
        'Win Rate': win_rate,
        'Win:Loss Ratio': win_loss_ratio,
        'Trades/Month': trades_per_month,
        'Absolute Quantity Counts': abs_quantity_counts,
        'Most Traded Symbol': most_traded_symbol,
        'Least Traded': least_traded_symbol,
        'Avg Losing PnL': avg_losing_pnl,
        'Avg Winning PnL': avg_winning_pnl,
        'Most Profitable': most_profitable,
        'Least Profitable': least_profitable,
        'Max Drawdown': max_drawdown,
        'Total Gain': total_gain,
        'Average Gain': avg_gain,
        'Biggest Investment': biggest_investment,
        'Average Position': avg_position,
        'Average Holdings': holdings,
        'Highest Traded Volume': highest_traded_volume,
        'Lowest Traded Volume': lowest_traded_volume,
        'Average Holding Days': average_holding_days,
    }

    portfolio_state['remaining'] = new_remaining
    portfolio_state['last_price'][ticker] = price

    df_row = pd.DataFrame([row], columns=COLUMNS)
    portfolio_state['portfolio_df'] = pd.concat(
        [portfolio_state['portfolio_df'], df_row],
        ignore_index=True
    )

    return row

    
def add_trade(ticker, action, position, price, quantity_buy, date=None, take_profit_pct=0.20, stop_loss_pct=0.10):
    process_trade(ticker, action, position, price, quantity_buy, date, take_profit_pct, stop_loss_pct)
    return get_portfolio_df()

In [83]:
reset_portfolio(200)
add_trade('aapl', 'hold', 'long', 10, 0, date='1/1/2025')
add_trade('aapl', 'buy', 'long', 10, 10, date='1/2/2025')
add_trade('aapl', 'hold', 'long', 11, 0, date='1/3/2025')
add_trade('aapl', 'hold', 'long', 12, 0, date='1/4/2025')
add_trade('aapl', 'sell', 'long', 12, 1, date='1/5/2025')
add_trade('aapl', 'hold', 'long', 13, 0, date='1/6/2025')
add_trade('msft', 'buy', 'long', 5, 5, date='1/7/2025')
add_trade('msft', 'hold', 'long', 6, 0, date='2/5/2025')
add_trade('aapl', 'buy', 'long', 13, 2, date='2/6/2025')
add_trade('aapl', 'sell', 'long', 13, 11, date='2/7/2025')
add_trade('aapl', 'sell', 'short', 12, 10, date='2/8/2025')  # Opening short
add_trade('msft', 'sell', 'long', 8, 1, date='2/9/2025')
add_trade('msft', 'buy', 'long', 9, 2, date='3/2/2025')
add_trade('msft', 'sell', 'long', 9, 6, date='3/3/2025')
add_trade('tsla', 'sell', 'short', 20, 5, date='3/4/2025')  # Opening short
add_trade('tsla', 'hold', 'short', 22, 0, date='3/5/2025')
add_trade('tsla', 'buy', 'short', 21, 4, date='3/6/2025')  # Closing short
add_trade('tsla', 'sell', 'short', 22, 1, date='3/7/2025')  # Adding to short
add_trade('tsla', 'buy', 'short', 24, 2, date='3/8/2025')  # Closing short

df = get_portfolio_df()
display(df)

C:\Users\dell\AppData\Local\Temp\ipykernel_13400\4201670646.py:1371: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  portfolio_state['portfolio_df'] = pd.concat(


,Date,Ticker,Buy/Sell,Position Taken,Current Position,Cash,Buyable/Sellable,Quantity Buy,Remaining,Current Quantity,Price,Avg Price,Cost Basis,Position Value PV,PnL (Long) Unrealized,PnL (Short) Unrealized,Pnl Unrealized,PnL Unrealized Value,PV (Long),PV (Short),Open Position,Open PV,Total PV,Equity: Total PV + Remaining,PnL Realized at Point of Time,PnL Realized Cummulative,Total PnL Overall (Unrealized+Realized),Daily PnL (Unrealized+Realized),Liquidation Price,Take Profit,Stop Loss,Last Day Pnl / Daily $,Daily %,Cumulative %,Performance,Trade No. (Position - Trade no. - Current Quantity),Total Trades,Win/Loss,Win Rate,Win:Loss Ratio,Trades/Month,Absolute Quantity Counts,Most Traded Symbol,Least Traded,Avg Losing PnL,Avg Winning PnL,Most Profitable,Least Profitable,Max Drawdown,Total Gain,Average Gain,Biggest Investment,Average Position,Average Holdings
0,1/1/2025,AAPL,Hold,Long,Hold,200,20.000000,0.0,200,0,10,0.000000,0.0,0,0.0,0.0,None,0.0,0.0,0.0,None,None,0.0,200.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.000000,0.0,0.0,No Buy/Sell,No Buy/Sell,None,NaN,0:0,0 (January),None,None,None,0.0,0.000000,None,None,0.000000,0.0,0.000000,None,0,0
1,1/2/2025,AAPL,Buy,Long,Long,200,10.000000,10.0,100.0,10.0,10,10.000000,100.0,100.0,0.0,0.0,AAPL 0.0,0.0,100.0,0.0,AAPL 10.0,AAPL 100.0,100.0,200.0,NaN,0.0,0.0,0.0,0.0,12.000000,9.000000,0.0,0.000000,0.0,0.0,Long - Buy - #1 Trade - 10,1 Trades,None,NaN,0:0,1 (January),AAPL 10,AAPL,AAPL,0.0,0.000000,None,None,0.000000,0.0,0.000000,AAPL: 100,100.0,1
2,1/3/2025,AAPL,Hold,Long,Long,200,9.090909,0.0,100.0,10.0,11,10.000000,100.0,110.0,10.0,0.0,AAPL 10.0,10.0,110.0,0.0,AAPL 10.0,AAPL 110.0,110.0,210.0,NaN,0.0,10.0,10.0,0.0,12.000000,9.000000,10.0,5.000000,5.0,5.0,Long - Hold - #1 Trade - 10,1 Trades,None,NaN,0:0,1 (January),AAPL 10,AAPL,AAPL,0.0,0.000000,None,None,0.090909,0.0,0.000000,AAPL: 100,0,1
3,1/4/2025,AAPL,Hold,Long,Long,200,8.333333,0.0,100.0,10.0,12,10.000000,100.0,120.0,20.0,0.0,AAPL 20.0,20.0,120.0,0.0,AAPL 10.0,AAPL 120.0,120.0,220.0,NaN,0.0,20.0,10.0,0.0,12.000000,9.000000,10.0,4.761905,10.0,10.0,Long - Hold - #1 Trade - 10,1 Trades,None,NaN,0:0,1 (January),AAPL 10,AAPL,AAPL,0.0,0.000000,None,None,0.166667,0.0,0.000000,AAPL: 100,0,1
4,1/5/2025,AAPL,Sell,Long,Long,200,9.333333,1.0,112.0,9.0,12,10.000000,90.0,108.0,18.0,0.0,AAPL 18.0,18.0,108.0,0.0,AAPL 9.0,AAPL 108.0,108.0,220.0,2.0,2.0,20.0,0.0,0.0,12.000000,9.000000,0.0,0.000000,10.0,10.0,Long - Sell - #1 Trade - 9,1 Trades,None,NaN,0:0,1 (January),AAPL 11,AAPL,AAPL,0.0,2.000000,AAPL 2.0,AAPL 2.0,0.166667,2.0,2.000000,AAPL: 100,0,1
5,1/6/2025,AAPL,Hold,Long,Long,200,8.615385,0.0,112.0,9.0,13,10.000000,90.0,117.0,27.0,0.0,AAPL 27.0,27.0,117.0,0.0,AAPL 9.0,AAPL 117.0,117.0,229.0,NaN,2.0,29.0,9.0,0.0,12.000000,9.000000,9.0,4.090909,14.5,14.5,Long - Hold - #1 Trade - 9,1 Trades,None,NaN,0:0,1 (January),AAPL 11,AAPL,AAPL,0.0,2.000000,AAPL 2.0,AAPL 2.0,0.166667,2.0,2.000000,AAPL: 100,0,1
6,1/7/2025,MSFT,Buy,Long,Long,200,17.400000,5.0,87.0,5.0,5,5.000000,25.0,25.0,0.0,0.0,"AAPL 27.0, MSFT 0.0",0.0,25.0,0.0,"AAPL 9.0, MSFT 5.0","AAPL 117.0, MSFT 25.0",142.0,229.0,NaN,2.0,29.0,0.0,0.0,6.000000,4.500000,0.0,0.000000,14.5,14.5,Long - Buy - #2 Trade - 5,2 Trades,None,NaN,0:0,2 (January),"AAPL 11, MSFT 5","AAPL, MSFT","MSFT, AAPL",0.0,2.000000,AAPL 2.0,AAPL 2.0,0.295775,2.0,1.000000,"AAPL: 100, MSFT: 25",62.5,2
7,2/5/2025,MSFT,Hold,Long,Long,200,14.500000,0.0,87.0,5.0,6,5.000000,25.0,30.0,5.0,0.0,"AAPL 27.0, MSFT 5.0",5.0,30.0,0.0,"AAPL 9.0, MSFT 5.0","AAPL 117.0, MSFT 30.0",147.0,234.0,NaN,2.0,34.0,5.0,0.0,6.000000,4.500000,5.0,2.183406,17.0,17.0,Long - Hold - #2 Trade - 5,2 Trades,None,NaN,0:0,1 (February),"AAPL 11, MSFT 5","AAPL, MSFT","MSFT, AAPL",0.0,2.000000,AAPL 2.0,AAPL 2.0,0.319728,2.0,1.000000,"AAPL: 100, MSFT: 25",0,2
8,2/6/2025,AAPL,Buy,Long,Long,200,4.692308,2.0,61.0,11.0,13,10.545455,116.0,143.0,27.0,0.0,"AAPL 27.000000000000007, MSFT 5.0",27.0,143.0,0.0,"AAPL 11.0, MSFT 5.0","AAPL 143.0, MSFT 30.0",173.0,234.0,NaN,2.0,34.0,0.0,0.0,1